In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [5]:
!ls 'drive/My Drive/Hackathon/Hackathon/SentimentAnalysis'

review_test.csv  review_train.csv


In [0]:
sent_train = pd.read_csv('drive/My Drive/Hackathon/Hackathon/SentimentAnalysis/review_train.csv')
sent_test = pd.read_csv('drive/My Drive/Hackathon/Hackathon/SentimentAnalysis/review_test.csv')

In [7]:
sent_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14825 entries, 0 to 14824
Data columns (total 3 columns):
Text         14825 non-null object
Score        14825 non-null int64
Sentiment    14825 non-null int64
dtypes: int64(2), object(1)
memory usage: 347.5+ KB


In [8]:
sent_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3707 entries, 0 to 3706
Data columns (total 3 columns):
Text         3707 non-null object
Score        3707 non-null int64
Sentiment    3707 non-null int64
dtypes: int64(2), object(1)
memory usage: 87.0+ KB


In [0]:
sent_train['src'] = 0
sent_test['src'] = 1

In [0]:
src = pd.concat([sent_train,sent_test])

In [11]:
src.head()

,Text,Score,Sentiment,src
0,"I got a free sample of these once, and now--we...",5,1,0
1,I used to get this Tea when I lived in Washing...,4,1,0
2,This is my all time favorite 'grab and go' sna...,5,1,0
3,This flavor is very good and unexpected. The ...,4,1,0
4,thrilled to have this assortment as i got the ...,4,1,0


In [12]:
src['Sentiment'].unique()

array([1, 0])

In [13]:
import unicodedata

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

import re

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text
  
!pip install contractions
!pip install textsearch
import contractions
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
wnl = WordNetLemmatizer()

def pos_tag_wordnet(tagged_tokens):
    tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wordnet.NOUN))
                            for word, tag in tagged_tokens]
    return new_tagged_tokens

def wordnet_lemmatize_text(text):
    tagged_tokens = nltk.pos_tag(nltk.word_tokenize(text))
    wordnet_tokens = pos_tag_wordnet(tagged_tokens)
    lemmatized_text = ' '.join(wnl.lemmatize(word, tag) for word, tag in wordnet_tokens)
    return lemmatized_text
  
def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

     |████████████████████████████████| 245kB 4.9MB/s 
     |████████████████████████████████| 317kB 46.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81708 sha256=3209c8a12ce12ef194d94a089ede0e2bf65a1cecdbbc19a4f8d50d7a9cc594d6
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
col = 'Text'
src[col] = src[col].map(lambda x: str(x).lower())
src[col] = src[col].map(lambda x: remove_accented_chars(x))
src[col] = src[col].map(lambda x: remove_special_characters(x))
src[col] = src[col].map(lambda x: contractions.fix(x))
src[col] = src[col].map(lambda x: wordnet_lemmatize_text(x))
src[col] = src[col].map(lambda x: remove_stopwords(x,True))

In [15]:
src

,Text,Score,Sentiment,src
0,get free sample nowwere subscribe save program...,5,1,0
1,use get tea live washington state miss niece t...,4,1,0
2,time favorite grab go snack bar leave sugar ru...,5,1,0
3,flavor good unexpected flavor cheese garlic go...,4,1,0
4,thrill assortment get chance choose another de...,4,1,0
5,love flavor coffee drawback set large cup want...,4,1,0
6,like last review mixture dog 2 chihuahua eat a...,5,1,0
7,wonderful handy product I thrill yummy way use...,5,1,0
8,great value sure place bag crowd transfer hard...,5,1,0
9,like flavor kind reminds jello pudding bite th...,4,1,0


In [0]:
train = src[src['src'] == 0]
test = src[src['src'] == 1]

**Dont use score below example use score and predicts 100%**

In [0]:
X_train = train[['Score','src']]
Y_train = train['Sentiment']
X_test = test[['Score','src']]
Y_test = test['Sentiment']

In [0]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report

In [19]:
from sklearn import  linear_model
mdl = linear_model.LogisticRegressionCV()
mdl.fit(X_train,Y_train)
Y_pred=mdl.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[[ 592    0]
 [   0 3115]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       592
           1       1.00      1.00      1.00      3115

    accuracy                           1.00      3707
   macro avg       1.00      1.00      1.00      3707
weighted avg       1.00      1.00      1.00      3707



In [0]:
X_train = train['Text']
Y_train = train['Sentiment']
X_test = test['Text']
Y_test = test['Sentiment']

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
#grid search result

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=0,max_df=1.0,use_idf=True, sublinear_tf=True) 
counts = vectorizer.fit_transform(X_train)
vocab = vectorizer.vocabulary_
classifier = SGDClassifier(alpha=1e-05,max_iter=50,penalty='elasticnet')
targets = Y_train
classifier = classifier.fit(counts, targets)
example_counts = vectorizer.transform(X_test)
Y_pred = classifier.predict(example_counts)

In [29]:
print(metrics.confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test, Y_pred))

[[ 374  218]
 [  89 3026]]
              precision    recall  f1-score   support

           0       0.81      0.63      0.71       592
           1       0.93      0.97      0.95      3115

    accuracy                           0.92      3707
   macro avg       0.87      0.80      0.83      3707
weighted avg       0.91      0.92      0.91      3707



In [0]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

In [31]:
for mdl in [ensemble.AdaBoostClassifier(),tree.DecisionTreeClassifier(),XGBClassifier(),naive_bayes.BernoulliNB()]:
   classifier = mdl  
   targets = Y_train
   classifier = classifier.fit(counts, targets)
   example_counts = vectorizer.transform(X_test)
   Y_pred = classifier.predict(example_counts)
   print(mdl.__class__.__name__) 
   print(metrics.confusion_matrix(Y_test,Y_pred))
   print(classification_report(Y_test, Y_pred))

AdaBoostClassifier
[[ 230  362]
 [  91 3024]]
              precision    recall  f1-score   support

           0       0.72      0.39      0.50       592
           1       0.89      0.97      0.93      3115

    accuracy                           0.88      3707
   macro avg       0.80      0.68      0.72      3707
weighted avg       0.86      0.88      0.86      3707

DecisionTreeClassifier
[[ 295  297]
 [ 370 2745]]
              precision    recall  f1-score   support

           0       0.44      0.50      0.47       592
           1       0.90      0.88      0.89      3115

    accuracy                           0.82      3707
   macro avg       0.67      0.69      0.68      3707
weighted avg       0.83      0.82      0.82      3707

XGBClassifier
[[ 127  465]
 [  14 3101]]
              precision    recall  f1-score   support

           0       0.90      0.21      0.35       592
           1       0.87      1.00      0.93      3115

    accuracy                           0.87  

**Word2Vec Model**

In [27]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder

2019-09-20 18:05:00,660 : INFO : 'pattern' package not found; tag filters are not available for English


In [0]:
le = LabelEncoder()
# tokenize train reviews & encode train labels
tokenized_train = [nltk.word_tokenize(text)
                       for text in X_train]
y_train = le.fit_transform(Y_train)
# tokenize test reviews & encode test labels
tokenized_test = [nltk.word_tokenize(text)
                       for text in X_test]
y_test = le.fit_transform(Y_test)

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [55]:
%%time
# build word2vec model
w2v_num_features = 500
w2v_model = gensim.models.Word2Vec(tokenized_train, size=w2v_num_features, window=150,
                                   min_count=10, workers=4, iter=5)

2019-09-20 18:32:57,523 : INFO : collecting all words and their counts
2019-09-20 18:32:57,525 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-20 18:32:57,602 : INFO : PROGRESS: at sentence #10000, processed 399863 words, keeping 23049 word types
2019-09-20 18:32:57,639 : INFO : collected 28994 word types from a corpus of 593315 raw words and 14825 sentences
2019-09-20 18:32:57,640 : INFO : Loading a fresh vocabulary
2019-09-20 18:32:57,661 : INFO : effective_min_count=10 retains 4246 unique words (14% of original 28994, drops 24748)
2019-09-20 18:32:57,662 : INFO : effective_min_count=10 leaves 544362 word corpus (91% of original 593315, drops 48953)
2019-09-20 18:32:57,680 : INFO : deleting the raw counts dictionary of 28994 items
2019-09-20 18:32:57,682 : INFO : sample=0.001 downsamples 58 most-common words
2019-09-20 18:32:57,683 : INFO : downsampling leaves estimated 478294 word corpus (87.9% of prior 544362)
2019-09-20 18:32:57,698 : INFO : esti

CPU times: user 44 s, sys: 67.4 ms, total: 44 s
Wall time: 22.6 s


In [56]:
w2v_model

In [0]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [0]:
# generate averaged word vector features from word2vec model
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

In [59]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape, ' Test features shape:', avg_wv_test_features.shape)

Word2Vec model:> Train features shape: (14825, 500)  Test features shape: (3707, 500)


ML Models

In [62]:
classifier = SGDClassifier(alpha=1e-05,max_iter=150,penalty='elasticnet')
classifier = classifier.fit(avg_wv_train_features, y_train)
Y_pred = classifier.predict(avg_wv_test_features)
print(metrics.confusion_matrix(y_test,Y_pred))
print(classification_report(y_test, Y_pred))

[[ 222  370]
 [ 191 2924]]
              precision    recall  f1-score   support

           0       0.54      0.38      0.44       592
           1       0.89      0.94      0.91      3115

    accuracy                           0.85      3707
   macro avg       0.71      0.66      0.68      3707
weighted avg       0.83      0.85      0.84      3707



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [132]:
for mdl in [ensemble.AdaBoostClassifier(n_estimators=150),tree.DecisionTreeClassifier(),XGBClassifier(n_estimator=200),naive_bayes.BernoulliNB()]:
   classifier = mdl  
   classifier = classifier.fit(avg_wv_train_features, y_train)
   Y_pred = classifier.predict(avg_wv_test_features)
   print(mdl.__class__.__name__) 
   print(metrics.confusion_matrix(y_test,Y_pred))
   print(classification_report(y_test, Y_pred))

AdaBoostClassifier
[[ 222  370]
 [ 161 2954]]
              precision    recall  f1-score   support

           0       0.58      0.38      0.46       592
           1       0.89      0.95      0.92      3115

    accuracy                           0.86      3707
   macro avg       0.73      0.66      0.69      3707
weighted avg       0.84      0.86      0.84      3707

DecisionTreeClassifier
[[ 249  343]
 [ 385 2730]]
              precision    recall  f1-score   support

           0       0.39      0.42      0.41       592
           1       0.89      0.88      0.88      3115

    accuracy                           0.80      3707
   macro avg       0.64      0.65      0.64      3707
weighted avg       0.81      0.80      0.81      3707

XGBClassifier
[[ 198  394]
 [ 115 3000]]
              precision    recall  f1-score   support

           0       0.63      0.33      0.44       592
           1       0.88      0.96      0.92      3115

    accuracy                           0.86  

FNN

In [0]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, input_shape=(num_input_features,)))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(1))
    dnn_model.add(Activation('sigmoid'))

    dnn_model.compile(loss='binary_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model

In [0]:
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

In [66]:
w2v_dnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

In [71]:
batch_size = 100
w2v_dnn.fit(avg_wv_train_features, y_train, epochs=10, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)

Train on 13342 samples, validate on 1483 samples
Epoch 1/50
13342/13342 [==============================] - 2s 117us/sample - loss: 0.2893 - acc: 0.8759 - val_loss: 0.3001 - val_acc: 0.8651
Epoch 2/50
13342/13342 [==============================] - 2s 118us/sample - loss: 0.2833 - acc: 0.8802 - val_loss: 0.2979 - val_acc: 0.8685
Epoch 3/50
13342/13342 [==============================] - 2s 117us/sample - loss: 0.2841 - acc: 0.8786 - val_loss: 0.2993 - val_acc: 0.8672
Epoch 4/50
13342/13342 [==============================] - 2s 115us/sample - loss: 0.2790 - acc: 0.8804 - val_loss: 0.3105 - val_acc: 0.8712
Epoch 5/50
13342/13342 [==============================] - 2s 116us/sample - loss: 0.2761 - acc: 0.8823 - val_loss: 0.3142 - val_acc: 0.8665
Epoch 6/50
13342/13342 [==============================] - 2s 116us/sample - loss: 0.2715 - acc: 0.8814 - val_loss: 0.3154 - val_acc: 0.8665
Epoch 7/50
13342/13342 [==============================] - 2s 116us/sample - loss: 0.2664 - acc: 0.8861 - val_lo

In [72]:
y_pred = w2v_dnn.predict_classes(avg_wv_test_features)
predictions = le.inverse_transform(y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.57      0.42      0.48       592
           1       0.90      0.94      0.92      3115

    accuracy                           0.86      3707
   macro avg       0.73      0.68      0.70      3707
weighted avg       0.84      0.86      0.85      3707

[[ 249  343]
 [ 189 2926]]


CNN

In [139]:
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [103]:
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(X_train)
t.word_index['<PAD>'] = 0
max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']

(('solubles', 28995), ('<PAD>', 0), 1)

In [0]:
train_sequences = t.texts_to_sequences(X_train)
test_sequences = t.texts_to_sequences(X_test)

In [107]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=28996
Number of Documents=14825


In [0]:
MAX_SEQUENCE_LENGTH = 1000

In [112]:
# pad dataset to a maximum review length in words
C_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
C_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
C_train.shape, C_test.shape

((14825, 1000), (3707, 1000))

In [0]:
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0

In [0]:
EMBED_SIZE = 300
EPOCHS=3
BATCH_SIZE=128

In [143]:
# create the model
model = Sequential()
model.add(Embedding(len(t.word_index), EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='elu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='elu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='elu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='elu')) 
model.add(Dense(128, activation='elu'))
model.add(Dense(64, activation='elu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1000, 300)         8698800   
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 1000, 128)         115328    
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 500, 128)          0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 500, 64)           24640     
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 250, 64)           0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 250, 32)           6176      
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 125, 32)         

In [144]:
%%time
# Fit the model
model.fit(C_train, y_train, 
          validation_split=0.1,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1)

Train on 13342 samples, validate on 1483 samples
Epoch 1/3
13342/13342 [==============================] - 262s 20ms/sample - loss: 0.4804 - acc: 0.8326 - val_loss: 0.3834 - val_acc: 0.8496
Epoch 2/3
13342/13342 [==============================] - 263s 20ms/sample - loss: 0.2221 - acc: 0.9098 - val_loss: 0.2163 - val_acc: 0.9117
Epoch 3/3
13342/13342 [==============================] - 263s 20ms/sample - loss: 0.0376 - acc: 0.9878 - val_loss: 0.3026 - val_acc: 0.9002
CPU times: user 25min 41s, sys: 13.8 s, total: 25min 55s
Wall time: 13min 9s


In [0]:
predictions = model.predict_classes(C_test)

In [148]:
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.64      0.66      0.65       592
           1       0.94      0.93      0.93      3115

    accuracy                           0.89      3707
   macro avg       0.79      0.80      0.79      3707
weighted avg       0.89      0.89      0.89      3707

[[ 392  200]
 [ 216 2899]]


LSTM

In [0]:
import tensorflow as tf

In [151]:
EMBEDDING_DIM = 300 # dimension for dense embeddings for each token
LSTM_DIM = 128 # total LSTM units

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(t.word_index), output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.SpatialDropout1D(0.1))
model.add(tf.keras.layers.CuDNNLSTM(LSTM_DIM, return_sequences=False))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1000, 300)         8698800   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 1000, 300)         0         
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 128)               220160    
_________________________________________________________________
dense_35 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 257       
Total params: 8,952,241
Trainable params: 8,952,241
Non-trainable params: 0
_________________________________________________________________


In [153]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [152]:
batch_size = 100
model.fit(C_train, y_train, epochs=2, batch_size=batch_size, 
          shuffle=True, validation_split=0.1, verbose=1)

Train on 13342 samples, validate on 1483 samples


InvalidArgumentError: ignored